## Fantasy AI Assistant

This notebook will get you on your way to a fantasy football championship. Start by importing the Agent class we're going to use, setting the config list, and giving some system instructions. Note that `gpt-4o-mini` seems to work well. This model will be using a lot of tokens, running `gpt-4-turbo` with this notebook will ring up a large bill quickly, even with the token compression implemented below.

In [4]:
from autogen import ConversableAgent
import os 

config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant that is an expert in tool use and in fantasy football. It is long past October of 2023, you need to check the current date to search for current news."
    "Before you recommend adding or dropping any player, search the web for any recent news regarding that player."
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

Creating the user proxy agent that acts as the stand in for you.

In [5]:
# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)


Now we start to create the tools that our assistant can use. There is potential for some change here, as these are written as tools rather than functions. When `autogen` hits their roadmap target of letting docker code execution agents take in user-defined functions, I will refactor this.

In [6]:
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from autogen import register_function

def google_it(query: str) -> str:
    "Search Google for keywords"
    #this is going to assume you have your environment variables set for GOOGLE_API and GOOGLE_CSE
    search = GoogleSearchAPIWrapper()

    google_search = Tool(
        name="google_search",
        description="Search Google for recent results.",
        func=search.run)
    
    return google_search({'query': query})

register_function(
    google_it,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="google_it",  # By default, the function name is used as the tool name.
    description="Searches google for recent results using the input as keywords",  # A description of the tool.
)


Here you need to make sure you've grabbed your `swid` and `s2` from ESPN's website. It's fairly straightforward when you go to developer tools after logging in. You also need to set up a `config.py` per the comment below the import. It needs to have a variable called `league_team_info` that contains a list of tuples. The values in the tuple must be your nickname for the league, the league ID, and your team's ID within the league. 

In [7]:
import os
from espn_api.football import League
from config import league_team_info
#[('work-friends',1234567,7)]
from espn_ff_toolkit import get_espn_leagues 


swid = os.environ['ESPN_SWID']
espn_s2 = os.environ['ESPN_S2']

year=2023
league_dict = get_espn_leagues(league_team_info,year,espn_s2=espn_s2,swid=swid)

In [8]:
from espn_ff_toolkit import get_rosters, get_free_agents, get_roster_and_projections

def get_rosters_tool()-> str:
    "Get the rosters for all current leagues"
    return get_rosters(league_dict)

register_function(
    get_rosters_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues"
)

def get_free_agents_tool() -> str:
    "Get the current free agents in all leagues"
    return get_free_agents(league_dict)

register_function(
    get_free_agents_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the current free agents in all leagues"
)

def get_roster_and_projections_tool() -> str:
    "Get the rosters for all current leagues and also all projected and past stats"
    return get_roster_and_projections(league_dict)

register_function(
    get_roster_and_projections_tool,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description = "Get the rosters for all current leagues and also all projected and past stats"
)

In [9]:
from datetime import datetime
def get_current_date() -> str:
    "Get the current date, because it is not October 2023"
    current_date = datetime.now()
    return str(current_date)

register_function(
    get_current_date,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    description="Find out what the current date is",  # A description of the tool.
)

In [ ]:
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages

llm_lingua = LLMLingua()
text_compressor = TextMessageCompressor(text_compressor=llm_lingua)
context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
context_handling.add_to_agent(assistant)

In [ ]:
chat_result = user_proxy.initiate_chat(assistant, message="What are my current rosters and projections, and are there free agents I should think about picking up?'")

In [12]:
from IPython.display import Markdown as md
md(chat_result.summary)

Here’s the current breakdown of rosters, projections, and available free agents from the leagues:

### Current Roster Projections

#### League: **squad**
1. **Ja'Marr Chase**
   - Points: 212.72
   - Projected Points: 0.0
2. **Davante Adams**
   - Points: 213.9
   - Projected Points: 0.0
3. **Josh Allen**
   - Points: 392.64
   - Projected Points: 0.0
4. **Rhamondre Stevenson**
   - Points: 126.7
   - Projected Points: 0.0
5. **Alvin Kamara**
   - Points: 195.5
   - Projected Points: 0.0
6. **Alexander Mattison**
   - Points: 118.2
   - Projected Points: 0.0
7. **George Kittle**
   - Points: 170.7
   - Projected Points: 0.0
8. **Courtland Sutton**
   - Points: 160.7
   - Projected Points: 0.0
9. **Jordan Addison**
   - Points: 186.3
   - Projected Points: 0.0

#### League: **work**
1. **Tyreek Hill**
   - Points: 316.9
   - Projected Points: 0.0
2. **Garrett Wilson**
   - Points: 165.7
   - Projected Points: 208.99
3. **Jalen Hurts**
   - Points: 356.82
   - Projected Points: 0.0
4. **Isiah Pacheco**
   - Points: 191.9
   - Projected Points: 0.0
5. **Raheem Mostert**
   - Points: 255.2
   - Projected Points: 0.0

### Current Free Agents

#### League: **squad**
- Tyler Bass
- Justin Herbert
- Cameron Dicker
- Mark Andrews
- Jason Myers
- Sam Howell
- Tyler Boyd
- Younghoe Koo
- Josh Downs
- Joe Burrow
- Tank Dell
- Gerald Everett
- Daniel Carlson
- Chigoziem Okonkwo
- Quentin Johnston
- Rondale Moore
- Deshaun Watson
- Derek Carr
- Dalvin Cook
- DJ Chark Jr.
- Zay Jones
- Treylon Burks
- JuJu Smith-Schuster
- Kirk Cousins
- Skyy Moore
- Darnell Mooney
- Nick Chubb
- Cam Akers
- Joshua Dobbs
- Mike Williams
- Cade Otton
- Anthony Richardson
- Juwan Johnson
- Jamaal Williams
- Graham Gano
- Cairo Santos
- Chris Boswell
- Jeff Wilson Jr.
- Tucker Kraft
- Daniel Jones
- Hunter Henry
- Tyler Conklin
- Keaton Mitchell
- J.K. Dobbins
- Aaron Rodgers
- Zach Ertz
- Gardner Minshew
- Riley Patterson
- Joe Flacco
- Justice Hill

#### League: **work**
- Evan McPherson
- Dolphins D/ST
- Christian Kirk
- Pat Freiermuth
- Khalil Herbert
- Matt Gay
- Cameron Dicker
- Zamir White
- Taysom Hill
- Falcons D/ST
- Jason Myers
- Miles Sanders
- Younghoe Koo
- Packers D/ST
- Jake Browning
- Tyler Higbee
- Logan Thomas
- Samaje Perine
- Quentin Johnston
- Rondale Moore
- D'Onta Foreman
- Deshaun Watson
- Clyde Edwards-Helaire
- Derek Carr
- Dalvin Cook
- Broncos D/ST
- Treylon Burks
- Kirk Cousins
- Roschon Johnson
- Skyy Moore
- Darnell Mooney
- Nick Chubb
- Cam Akers
- Joshua Dobbs
- Mike Williams
- Cade Otton
- Anthony Richardson
- Jamaal Williams
- Graham Gano
- Commanders D/ST
- Jeff Wilson Jr.
- Tucker Kraft
- Daniel Jones
- Hunter Henry
- Bengals D/ST
- Elijah Mitchell
- Keaton Mitchell
- J.K. Dobbins
- Zach Ertz
- Gardner Minshew

### Considerations for Free Agents
You might want to consider targeting players that fill gaps in your team or have a strong projection for upcoming games. Players like **Mark Andrews** and **Christian Kirk** could be valuable pickups if you need a tight end or wide receiver, respectively. Additionally, **Dalvin Cook** and **Nick Chubb** could provide solid running back depth.

Let me know if you need any more specific insights or analyses! 

****